# Computer and Robot Vision project (Teil 1: Die Lokalisierung)

In diesem Notebook geht es darum, wie wir Geschwindigkeitsschilder lokalisieren und klassifizieren können. Anders als üblich setzen wir dabei nicht auf herkömmliche Bildverarbeitung, sondern trainieren KI-Modelle, die diese Aufgaben eigenständig übernehmen können.

Die Idee dahinter ist, das Projekt in zwei Teile aufzuteilen, die unabhängig voneinander funktionieren können. Ein Modell wird speziell darauf trainiert, die Schilder im Bild zu lokalisieren, während ein separates Modell nur für die Klassifizierung zuständig ist. Dieses Notebook konzentriert sich auf die Lokalisierung und gibt Einblicke in die Methoden und Erkenntnisse, die dabei zum Einsatz kommen.

---

**Author:**

Dominik Bücher, Hochschuhle Heilbronn, Automotive System Engineering Master | dbuecher@stud.hs-heilbronn.de

Aaron Kiani, Hochschuhle Heilbronn, Mechatronik und Robotik Master | akiani@stud.hs-heilbronn.de


**Professor:**

Prof. Dr. Dieter Maier 

dieter.maier@hs-heilbronn.de

---

## Table of Contents

#### 1. [Einleitung](#Introduction)
#### 2. [Importieren der Bibliotheken](#Import)
#### 3. [Erstellen eines eigenen Datensatzes](#Dataset)
#### 4. [Vorstellen der verwendeten Modelle](#modelle)
#### 5. [Training](#Training)
#### 6. [Evaluation](#Evaluation)
#### 7. [Testing](#Testing)
#### 8. [Diskussion der Ergebnisse](#Evaluation)

---

## 1. Einleitung <a id="Introduction"></a>

Im kommenden Kapitel wird zunächst die Struktur dieses Notebooks vorgestellt sowie das übergeordnete Ziel dieser Aufgabe erläutert.

Wie bereits zu Beginn des Notebooks erwähnt, liegt der Fokus dieses Abschnitts auf der Lokalisierung von Geschwindigkeitsschilder. Hierbei stehen verschiedene Ansätze zur Verfügung, wobei wir uns für einen eher klassischen Weg entschieden haben: die Verwendung von Cascading-Modellen.

Cascading-Modelle repräsentieren eine spezielle Klasse von Klassifikationsmodellen, die nicht nur auf dem etablierten "Haar-like features"-Ansatz basieren, sondern auch Local Binary Pattern (LBP) einbeziehen. Diese kombinierte Methode vereint die Effizienz der Haar-ähnlichen Merkmale mit der präzisen Texturerkennung von LBP, was eine vielseitige Lösung für die Objekterkennung in Bildern bietet. Der Begriff "cascading" (kaskadierend) leitet sich davon ab, dass diese Modelle hierarchisch angeordnete Kaskaden von Klassifikatoren nutzen.

Die Grundidee der Cascading-Modelle besteht darin, hierarchisch angeordnete Kaskaden von Klassifikatoren zu verwenden. Jede Stufe dieser Kaskade filtert gezielt bestimmte Bildbereiche und leitet nur die vielversprechendsten Regionen an die nachfolgende Stufe weiter. Diese kaskadierende Struktur ermöglicht eine effiziente Ablehnung von irrelevanten Bereichen und konzentriert die Ressourcen auf vielversprechende Regionen, was insbesondere bei der Verarbeitung großer Bildmengen von Vorteil ist.

Obwohl Cascading-Modelle ihre Bekanntheit hauptsächlich der Gesichtserkennung verdanken, eignen sie sich ebenso für die Erkennung anderer Objekte wie Geschwindigkeitsschilder oder Fahrzeuge. Der Trainingsprozess umfasst die Nutzung positiver und negativer Beispiele, um das Modell auf die spezifischen Mustererkennungsaufgaben anzupassen. Diese adaptive Anpassung ermöglicht eine präzise und effiziente Objekterkennung in verschiedenen Kontexten.

---

## 2. Importieren der Bibliotheken <a id="Import"></a>

In [1]:
import os
import shutil
import cv2
import numpy as np
import tensorflow as tf
import time
import re
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from PIL import Image
import csv
import random

---
## 3. Erstellen eines eigenen Datensatzes <a id="Dataset"></a>

Um ein Cascading-Modell erfolgreich zu trainieren, ist ein geeigneter Datensatz von entscheidender Bedeutung. Zu diesem Zweck wurden verschiedene Videos in und um Heilbronn aufgenommen. Die Aufnahmen erfolgten mit einer GoPro 11 unter den folgenden Einstellungen: 30 FPS (Bildern pro Sekunde) und einer Auflösung von 1920x1080 Pixel (HD).

Die aufgenommenen Videos wurden anschließend mithilfe eines speziellen Skripts in Bilddateien umgewandelt.

In [ ]:
def video_to_frames(video_path, output_path):
    video_capture = cv2.VideoCapture(video_path)
    
    if not video_capture.isOpened():
        print("Fehler beim Öffnen des Videos.")
        return
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    frame_count = 0
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        frame_filename = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(output_path, frame_filename)
        cv2.imwrite(frame_path, frame)
        frame_count += 1
    print(f"{frame_count} Frames wurden erfolgreich extrahiert.")
    video_capture.release()


video_path = r"videos_24_11_2023\video_speed_combined.mp4"
output_path = r"videos_24_11_2023\combined"
video_to_frames(video_path, output_path)

Nach der Umwandlung der Videos in Bilder ist es nun möglich, die einzelnen Bilder zu labeln, was von entscheidender Bedeutung ist, da ein Modell ohne diese Annotationen nicht trainiert werden kann.

Der Kennzeichnungsprozess beinhaltete das Auffinden aller Geschwindigkeitsschilder in einem Bild und das Hinzufügen von Begrenzungsrahmen (Bounding Boxes) um diese. Es gibt verschiedene Programme, die für diese Aufgabe verwendet werden können, und wir haben uns für DarkLabel 2.4 entschieden. Dabei ist zu erwähnen das das Labeln sehr viel zeit in anspruch nehmen kann, wobei DarkLabel2.4 diese zeit zumindest etwas verkürzt hat. 

Nachdem alle Bilder erfolgreich gelabelt wurden, wird eine Textdatei generiert, die die Daten zu jeder einzelnen Bounding Box enthält. Diese Datei bildet die Grundlage für das spätere Training des Modells.

Die Text Datei ist dabei wie folgt aufgebaut: 00000.jpg,1,795,428,50,49. Jedoch wird für das training ein Format wie dieses "00000.jpg 1 795 428 50 49" benötigt, bzw "frame_3321.jpg 2 177 499 157 141 1692 577 103 93" falls es zwei Bounding Boxen in einem Bild gibt. Zusätzlich dazu dürfen die Koordinaten nicht die Größe des Bildes überschreiten. All diese kleinen Änderungen werden in den nachfolgenden Funktionen behoben.

In [ ]:
def adjust_coordinates(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            parts = line.strip().split(' ')
            if len(parts) == 6:
                x_coordinate = int(parts[2])
                width = int(parts[4])

                # Überprüfe, ob die Koordinaten größer als 1919 sind
                if x_coordinate > 1919 or (x_coordinate + width) > 1919:
                    if x_coordinate > 1919:
                        x_coordinate = 1919 - width
                    if (x_coordinate + width) > 1919:
                        width = 1919 - x_coordinate

                # Überprüfe, ob die Koordinaten kleiner als 1 sind
                if x_coordinate < 1 or (x_coordinate + width) < 1:
                    if x_coordinate < 1:
                        x_coordinate = 1
                    if (x_coordinate + width) < 1:
                        width = 1 - x_coordinate

                # Erstelle die aktualisierte Zeile
                updated_line = f"{parts[0]},{parts[1]},{x_coordinate},{parts[3]},{width},{parts[5]}\n"
                outfile.write(updated_line)
            else:
                # Schreibe Zeilen ohne das erwartete Format unverändert
                outfile.write(line)


def combine_lines(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        lines = infile.readlines()
        i = 0
        while i < len(lines) - 1:
            current_line = lines[i].strip().split(';')
            next_line = lines[i + 1].strip().split(';')

            if current_line[0] == next_line[0]:
                combined_line = [current_line[0], str(int(current_line[1]) + 1)] + [current_line[2]] + current_line[3:] + next_line[1:]
                lines[i] = ' '.join(combined_line) + '\n'
                lines.pop(i + 1)
            else:
                i += 1

        # Ersetze alle Kommas durch Leerzeichen in den verbleibenden Zeilen
        for j in range(len(lines)):
            lines[j] = lines[j].replace(';', ' ')

        outfile.writelines(lines)



adjust_coordinates(r'Dominik\annotation_files\train.txt', r'Dominik\annotation_files\train2.txt')
combine_lines(r'Dominik\gt copy2.txt', r'Dominik\pos_new.txt')

Nachdem die Textdatei nun im korrekten Format vorliegt, schreiten wir zum nächsten Schritt: der Aufteilung des Datensatzes in Trainings- und Testbilder. Der folgende Code generiert zwei neue Textdateien für diese Aufteilung. Dabei werden die Daten zunächst zufällig durchmischt und anschließend aufgeteilt. Der Trainingsdatensatz umfasst dabei 95% der Bilder, während der Testdatensatz nur 5% enthält. Obwohl eine übliche Aufteilung 90 zu 10 wäre, haben wir uns aufgrund der begrenzten Anzahl von insgesamt 3726 Bildern entschieden, dem Training einen größeren Datensatzanteil zuzuweisen, wie in der folgenden Abbildung verdeutlicht wird:

<img src="Notebook_Bilder/Split_data.png" width="500" height="400">


In [ ]:
def split_dataset(input_datei, ausgabe_datei_1, ausgabe_datei_2, trennungsprozentsatz=5):
    with open(input_datei, 'r') as datei:
        zeilen = datei.readlines()
    
    random.shuffle(zeilen)
    
    trennungspunkt = int(len(zeilen) * (trennungsprozentsatz / 100))
    ausgabe_1 = zeilen[:trennungspunkt]
    ausgabe_2 = zeilen[trennungspunkt:]
    
    with open(ausgabe_datei_1, 'w') as datei_1:
        datei_1.writelines(ausgabe_1)
    
    with open(ausgabe_datei_2, 'w') as datei_2:
        datei_2.writelines(ausgabe_2)


input_datei = r'Dominik\annotation_files\pos_own.txt'
ausgabe_datei_1 = r'Dominik\annotation_files\test.txt'
ausgabe_datei_2 = r'Dominik\annotation_files\train.txt'
split_dataset(input_datei, ausgabe_datei_1, ausgabe_datei_2, trennungsprozentsatz=5)

Nachdem die beiden Textdateien erstellt wurden, haben Sie die Möglichkeit, die Bilder in die entsprechenden Ordner zu verschieben. Obwohl dies nicht zwingend erforderlich ist, trägt es zur besseren Übersicht bei. Für das eigentliche Training muss die Textdatei der Trainingsdaten zwar erneut verarbeitet werden, doch dieser Schritt wird in einem späteren Kapitel ausführlich behandelt.

---

## 4. Vorstellen der verwendeten Modelle <a id="modelle"></a>